# ETL1 - SCRIPT DE PIPELINE DE DADOS

##### Script responsável por baixar os dados do CENSO ESCOLAR 2019, CENSO ESCOLAR 2020 e COVID19 e salvá-los no Cloud Storage

In [ ]:
# Baixar e atualizar as bibliotecas necessárias
# Necessário executar apenas uma vez
!pip install --upgrade pandas
!pip install --upgrade numpy
!pip install --upgrade requests
!pip install --upgrade shutil
!pip install --upgrade gzip
!pip install --upgrade google-cloud-storage

In [ ]:
# Importa as bibliotecas necessárias
import pandas as pd
import numpy as np
import zipfile
import gzip
import requests
from io import BytesIO
import os
import shutil
from google.cloud import storage

In [ ]:
# Cria a conexão com o bucket do GCP e define o bucket a ser usado
storage_client = storage.Client()
bucket = storage_client.bucket('igti-pa')

In [ ]:
# Define as variáveis do script - diretório de dados, url dos dados do censo escolar e do covid
dir_local = '/dados/'
url_covid = "https://data.brasil.io/dataset/covid19/caso_full.csv.gz"
url_censo_2019 = 'https://download.inep.gov.br/microdados/microdados_educacao_basica_2019.zip'
url_censo_2020 = 'https://download.inep.gov.br/dados_abertos/microdados_censo_escolar_2020.zip'

In [ ]:
# Cria o diretório temporário no qual serão salvos os arquivos localmente
try:
    os.mkdir(dir_local)
except OSError as e:
    print(e)
else:
    print("O diretório '{}' foi criado com sucesso!".format(dir_local))

In [ ]:
# Realiza o download dos dados do COVID19 
try:
    filename_covid = dir_local + url_covid.split("/")[-1]
    with open(filename_covid, "wb") as f:
        r = requests.get(url_covid)
        f.write(r.content)
    print("Dados do COVID19 baixados com sucesso!")
except:
    print("Ocorreu um erro ao baixar os dados do COVID19!")

In [ ]:
# Realiza o upload dos dados do COVID19 para o bucket do GCP
try:  
    blob_covid = bucket.blob('bronze/covid19.gz')
    blob_covid.upload_from_filename(filename_covid)
    print("O upload dos dados do COVID19 foi concluído com sucesso!")
except:
    print("Ocorreu um erro ao realizar o upload dos dados do COVID19!")

In [ ]:
# Realiza o download dos dados do CENSO 2019 
try:
    filebytes = BytesIO (requests.get(url_censo_2019).content)
    myzip = zipfile.ZipFile(filebytes)
    myzip.extractall(dir_local)
    print("Dados do CENSO 2019 baixados com sucesso!")
except:
    print("Ocorreu um erro ao baixar os dados do CENSO 2019!")

In [ ]:
# Realiza o upload dos arquivos CSV do CENSO 2019 para o bucket do GCP
try:
    censo_2019 = dir_local + 'microdados_educacao_basica_2019/DADOS/'
    for diretorio, subpastas, arquivos in os.walk(censo_2019):
        for arquivo in arquivos:
            if '.CSV' in arquivo:
                blob = bucket.blob('bronze/censo_2019/'+arquivo)                
                blob.upload_from_filename(diretorio+arquivo)
                print(arquivo + 'carregado com sucesso no GCP')      
    print("O upload dos dados do CENSO 2019 foi concluído com sucesso!")
except:
    print("Ocorreu um erro ao realizar o upload dos dados do CENSO 2019!")

In [ ]:
# Realiza o download dos dados do CENSO 2020
try:
    filebytes = BytesIO (requests.get(url_censo_2020).content)
    myzip = zipfile.ZipFile(filebytes)
    myzip.extractall(dir_local)
    print("Dados do CENSO 2020 baixados com sucesso!")
except:
    print("Ocorreu um erro ao baixar os dados do CENSO 2020!")

In [ ]:
# Realiza o upload dos arquivos CSV do CENSO 2020  para o bucket do GCP
try:
    censo_2020 = dir_local + 'microdados_educacao_basica_2020/DADOS/'
    for diretorio, subpastas, arquivos in os.walk(censo_2020):
        for arquivo in arquivos:
            if '.CSV' in arquivo:
                blob = bucket.blob('bronze/censo_2020/'+arquivo)                
                blob.upload_from_filename(diretorio+arquivo)
                print(arquivo + ' carregado com sucesso no GCP')      
    print("O upload dos dados do CENSO 2020 foi concluído com sucesso!")
except:
    print("Ocorreu um erro ao realizar o upload dos dados do CENSO 2020!")

In [ ]:
# Deleta o diretório temporário no qual estão salvos os arquivos baixados
try:
    shutil.rmtree(dir_local)
except OSError as e:
    print(e)
else:
    print("O diretório '{}' foi deletado com sucesso!".format(dir_local))

### FIM DO SCRIPT ETL1